In [2]:
from svgpathtools import svg2paths, wsvg
from numpy import *

ModuleNotFoundError: No module named 'svgpathtools'

In [2]:
paths, attributes = svg2paths('drawing.svg')
[a["id"] for a in attributes]

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'colon']

In [6]:
def getVecs(ps):
    a = set()
       
    for p in ps:
        a.add(p.start)
        a.add(p.end)
    a = array(list(a))
    
    lineInds = zeros((len(ps), 2), dtype=int)
    for i, p in enumerate(ps):
        lineInds[i, 0] = where(abs(a - p.start) == 0)[0][0]
        lineInds[i, 1] = where(abs(a - p.end) == 0)[0][0]
    
#     a -= ps[0].start
    a -= mean(a)
    return a, lineInds

getVecs(paths[0])

(array([-2.63669587-2.34105293j, -1.44251981-1.17746293j,
         2.63449829-2.34105293j,  1.44406039-1.17746293j,
        -1.44200081+1.25440007j, -2.63590087+2.26411578j,
         1.44406039+1.25440007j,  2.63449829+2.26411578j]),
 array([[1, 3],
        [3, 6],
        [6, 4],
        [4, 1],
        [0, 2],
        [2, 7],
        [7, 5],
        [5, 0]]))

In [7]:
def extrude(vs, ls, depth=3):
    n_vs = vs.shape[0]
    vs_ = zeros((n_vs * 2, 3), dtype=float)

    # bottom side at z = 0
    vs_[:n_vs, 0] = real(vs)
    vs_[:n_vs, 1] = imag(vs)
    vs_[:n_vs, 2] = -depth / 2

    # offset top side at z = depth
    vs_[n_vs:, :] = vs_[:vs.shape[0], :]
    vs_[n_vs:, 2] = depth / 2

    # add edges
    n_ls = ls.shape[0]
    ls_ = zeros((n_ls * 2 + n_vs, 2), dtype=int)
    # edges bottom side
    ls_[:n_ls, :] = ls
    # edges top side
    ls_[n_ls: n_ls * 2, :] = ls + n_vs
    # connect them
    ls_[n_ls * 2:, 0] = arange(n_vs)
    ls_[n_ls * 2:, 1] = arange(n_vs) + n_vs
    return vs_, ls_

'16, 8, 16, 24, 20, 16, 20, 12, 24, 20, 16'

In [99]:
def toHeader(vs, ls, lvs=None, les=None):
    s = '''
namespace numbers
{{
    const int vertexCount = {vcount};
    const int edgeCount = {ecount};
    const int triangleCount = 0;
    const float vertices[][3] = {{
'''.format(vcount=len(vs), ecount=len(ls))
    for v in vs:
        s += '    {0:12.6f}, {1:12.6f}, {2:12.6f},\n'.format(*v)
    s = s[:-2] + '\n    };\n'
    s += '    const unsigned short edges[][2] = {\n'
    for l in ls:
        s += '    {:6d}, {:6d},\n'.format(l[0], l[1])
    s = s[:-2] + '\n    };\n'
    
    if lvs:
        tmp = '    const uint8_t lvs[] = {'
        for lv in lvs:
            tmp += '{:d}, '.format(lv)
        s += tmp[:-2] + '};\n'
    if les:
        tmp = '    const uint8_t les[] = {'
        for le in les:
            tmp += '{:d}, '.format(le)
        s += tmp[:-2] + '};\n'
    s += '}\n'
    return s

In [100]:
verts = list()
edgs = list()
lvs = list()
les = list()
lv, le = 0, 0
for ps in paths:
    v, e = extrude(*getVecs(ps), depth=2)
    lv = v.shape[0]
    le = e.shape[0]
    lvs.append(lv)
    les.append(le)
    verts.append(v)
    edgs.append(e)

In [101]:
verts = vstack(verts)
edgs = vstack(edgs)

In [102]:
s = toHeader(verts, edgs, lvs, les)
print(s)


namespace number
{
    const int vertexCount = 192;
    const int edgeCount = 288;
    const int triangleCount = 0;
    const float vertices[][3] = {
       -2.636696,    -2.341053,    -1.000000,
       -1.442520,    -1.177463,    -1.000000,
        2.634498,    -2.341053,    -1.000000,
        1.444060,    -1.177463,    -1.000000,
       -1.442001,     1.254400,    -1.000000,
       -2.635901,     2.264116,    -1.000000,
        1.444060,     1.254400,    -1.000000,
        2.634498,     2.264116,    -1.000000,
       -2.636696,    -2.341053,     1.000000,
       -1.442520,    -1.177463,     1.000000,
        2.634498,    -2.341053,     1.000000,
        1.444060,    -1.177463,     1.000000,
       -1.442001,     1.254400,     1.000000,
       -2.635901,     2.264116,     1.000000,
        1.444060,     1.254400,     1.000000,
        2.634498,     2.264116,     1.000000,
       -0.587188,     2.305428,    -1.000000,
        0.591660,     2.305428,    -1.000000,
       -0.596131,    

In [104]:
def sInd(n):
    si = 0
    for i in range(n):
        si += lvs[i]
    return si

In [110]:
sInd(1)

16